In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import urllib.request as urllib2
import pandas as pd
from os import listdir
from os.path import isfile, join

## GialloZafferano Corpus

In this section we are going to create a web scraper to retrieve Italian dishes recipes from GialloZafferano (https://www.giallozafferano.it/), one of the most popular cooking websites in Italy. 

In [2]:
#creating a function that converts a list to a string
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s))

In [3]:
def get_ingredients(url):
    
    #retrieving the url in json format
    myurl = urllib2.urlopen(url).read()
    soup = BeautifulSoup(myurl)
    ingredients = json.loads(soup.find('script', type='application/ld+json').text)
    
    #cleaning the text:
    ingredients = str(ingredients)
    ingredients = ingredients.split('[') 
    ingredients = ingredients[1:3] 
    ingredients = [i.split(']', 1)[0] for i in ingredients] 
    
    ingredients = listToString(ingredients)
    
    ingredients = re.sub('[^a-zA-ZÀ-ÿ.\s]', '', ingredients) 
    
    ingredients = ingredients.lower() 
       
    return ingredients

#testing that the function works properly 
get_ingredients("https://ricette.giallozafferano.it/Gelato-al-pistacchio-senza-gelatiera.html")

'latte intero  g panna fresca liquida  g acqua  g zucchero  g pistacchi sgusciati  g per preparare il gelato al pistacchio senza gelatiera partite dallo sciroppo in un pentolino versate lo zucchero   lacqua  e accendete la fiamma a fuoco basso  .xa nel frattempo in una caraffa versate sia il latte che la panna  . non appena lo sciroppo sarà arrivato a bollore   spegnete il fuoco e lasciatelo intiepidire. versate lo sciroppo allinterno del composto di latte e panna  e frullate il tutto con un mixer ad immersione  . trasferite allinterno di uno stampo di silicone  o negli stampini per il ghiaccio. trasferite in freezer e lasciatelo gelare per almeno  notte  .xa a questo punto estraete i cubetti dallo stampo  e trasferiteli allinterno di un robot da cucina  . frullate il tutto fino ad ottenere un composto cremoso  xa aggiungete i pistacchi  e frullate ancora fino ad ottenere una crema liscia  . trasferite in un contenitore meglio di alluminio in ceramica o vetro. evitate la plastica  e tr

In [6]:
#creating a list of recipes where we will store the recipes we will use
recipes = ["https://ricette.giallozafferano.it/Spaghetti-alla-Carbonara.html", "https://ricette.giallozafferano.it/Spaghetti-all-Amatriciana.html", 
           "https://ricette.giallozafferano.it/Risotto-allo-Zafferano.html", "https://ricette.giallozafferano.it/Spezzatino-di-manzo.html", 
          "https://ricette.giallozafferano.it/Tiramisu.html", "https://ricette.giallozafferano.it/Pasta-al-forno.html",
          "https://ricette.giallozafferano.it/Paccheri-ripieni-con-salsiccia-e-funghi.html", "https://ricette.giallozafferano.it/Spaghetti-allo-scoglio.html",
          "https://ricette.giallozafferano.it/Pasta-gorgonzola-e-noci.html", "https://ricette.giallozafferano.it/Parmigiana-di-melanzane.html",
          "https://ricette.giallozafferano.it/Polpettine-di-tonno-e-ricotta.html", "https://ricette.giallozafferano.it/Filetto-al-pepe-verde.html",
          "https://ricette.giallozafferano.it/Zucchine-ripiene.html", "https://ricette.giallozafferano.it/Saltimbocca-alla-Romana.html",
          "https://ricette.giallozafferano.it/Rotolo-di-frittata-farcito.html", "https://ricette.giallozafferano.it/Scaloppine-ai-funghi.html",
          "https://ricette.giallozafferano.it/Tiramisu.html", "https://ricette.giallozafferano.it/Tortino-di-cioccolato-con-cuore-fondente.html", 
          "https://ricette.giallozafferano.it/Crema-pasticcera.html", "https://ricette.giallozafferano.it/Torta-rustica-di-mele.html", 
          "https://ricette.giallozafferano.it/Crostata-alla-frutta.html", "https://ricette.giallozafferano.it/Crostata-morbida-al-cioccolato.html", 
          "https://ricette.giallozafferano.it/Cannoli-siciliani.html", "https://ricette.giallozafferano.it/Torta-margherita.html", 
          "https://ricette.giallozafferano.it/Bigne-alla-crema.html", "https://ricette.giallozafferano.it/Taralli.html", 
          "https://ricette.giallozafferano.it/Arancini-di-riso.html", "https://ricette.giallozafferano.it/Mozzarella-in-carrozza.html",
          "https://ricette.giallozafferano.it/Focaccia-con-pomodorini-e-origano.html", "https://ricette.giallozafferano.it/Grissini.html",
          "https://ricette.giallozafferano.it/Frittata-di-zucchine.html", "https://ricette.giallozafferano.it/Involtini-di-melanzane.html",
          "https://ricette.giallozafferano.it/Insalata-di-polpo-prezzemolata.html", "https://ricette.giallozafferano.it/Asparagi-in-sfoglia-con-crudo.html",
          "https://ricette.giallozafferano.it/Carciofi-ripieni.html", "https://ricette.giallozafferano.it/Patate-al-forno.html",
          "https://ricette.giallozafferano.it/Sformato-di-verdure.html", "https://ricette.giallozafferano.it/Verdure-gratinate-al-forno.html",
          "https://ricette.giallozafferano.it/Peperonata.html", "https://ricette.giallozafferano.it/Cipolline-in-agrodolce.html",
          "https://ricette.giallozafferano.it/Polpettone-alla-ligure.html", "https://ricette.giallozafferano.it/Insalata-di-ceci-gamberi-e-rucola.html",
          "https://ricette.giallozafferano.it/Insalata-con-avocado.html", "https://ricette.giallozafferano.it/Fagiolini-alla-pugliese.html",
          "https://ricette.giallozafferano.it/Panzerotti-calzoni-fritti.html", "https://ricette.giallozafferano.it/Treccia-di-pasta-lievitata.html",
          "https://ricette.giallozafferano.it/Pizza-Margherita.html", "https://ricette.giallozafferano.it/Focaccia-col-formaggio.html",
          "https://ricette.giallozafferano.it/Pane-ai-cereali.html", "https://ricette.giallozafferano.it/Risotto-alla-salsiccia.html",
          "https://ricette.giallozafferano.it/Risotto-ai-funghi.html", "https://ricette.giallozafferano.it/Risotto-radicchio-e-pancetta.html",
          "https://ricette.giallozafferano.it/Aragosta-allo-Champagne-con-riso-pilaf.html", "https://ricette.giallozafferano.it/Astice-alla-Catalana.html",
          "https://ricette.giallozafferano.it/Pesce-spada-alla-siciliana.html", "https://ricette.giallozafferano.it/Pesce-spada-in-salmoriglio.html",
          "https://ricette.giallozafferano.it/Filetti-di-merluzzo-al-forno.html", "https://ricette.giallozafferano.it/Filetto-di-merluzzo-allo-zenzero.html",
          "https://ricette.giallozafferano.it/Riso-venere-ai-frutti-di-mare.html", "https://ricette.giallozafferano.it/Strozzapreti-ai-frutti-di-mare.html",
          "https://ricette.giallozafferano.it/Carpaccio-di-polpo.html", "https://ricette.giallozafferano.it/Orata-al-forno.html", 
          "https://ricette.giallozafferano.it/Branzino-al-forno.html",  "https://ricette.giallozafferano.it/Baccala-al-forno.html",
          "https://ricette.giallozafferano.it/Totani-ripieni-al-forno.html", "https://ricette.giallozafferano.it/Spaghetti-con-le-acciughe-e-il-pangrattato.html",
          "https://ricette.giallozafferano.it/Involtini-di-vitello-con-prosciutto-e-formaggio.html", "https://ricette.giallozafferano.it/Costolette-d-agnello-al-forno.html",
          "https://ricette.giallozafferano.it/Agnello-gratinato-con-carciofi-e-piselli.html", "https://ricette.giallozafferano.it/Frittatine-di-salmone-al-timo-e-taccole.html",
          "https://ricette.giallozafferano.it/Insalata-di-finocchi-e-arance.html", "https://ricette.giallozafferano.it/Insalata-di-finocchi-e-arance.html",
          "https://ricette.giallozafferano.it/Insalata-con-scamorza-e-verdure-grigliate.html", "https://ricette.giallozafferano.it/Insalata-di-ceci-estiva.html",
          "https://ricette.giallozafferano.it/Limoncello.html", "https://ricette.giallozafferano.it/Marmellata-di-limoni.html", 
          "https://ricette.giallozafferano.it/Marmellata-di-limoni.html", "https://ricette.giallozafferano.it/Negroni-cocktail.html",
          "https://ricette.giallozafferano.it/Pasta-ai-4-formaggi-e-speck.html", "https://ricette.giallozafferano.it/Torta-rustica-ai-4-formaggi.html",
          "https://ricette.giallozafferano.it/Fagottini-di-porri-salmone-e-robiola.html", "https://ricette.giallozafferano.it/Linguine-al-nero-di-seppia.html",
          "https://ricette.giallozafferano.it/Gelato-alla-stracciatella.html", "https://ricette.giallozafferano.it/Gelato-al-pistacchio-senza-gelatiera.html",
          "https://ricette.giallozafferano.it/Spaghetti-alla-Norma.html", "https://ricette.giallozafferano.it/Scialatelli-con-la-nduja.html",
          "https://ricette.giallozafferano.it/Petto-d-anatra-al-balsamico.html", "https://ricette.giallozafferano.it/Agnolotti.html",
          "https://ricette.giallozafferano.it/Cannelloni-di-melanzane-alla-birra.html", "https://ricette.giallozafferano.it/Ossobuco-alla-milanese-con-risotto-giallo.html",
          "https://ricette.giallozafferano.it/Baccala-con-uvetta-e-pinoli.html", "https://ricette.giallozafferano.it/Orecchiette-alle-erbe-aromatiche.html",
          "https://ricette.giallozafferano.it/Gallinella-in-guazzetto.html", "https://ricette.giallozafferano.it/Ciambella-al-caffe.html"]

In [7]:
len(recipes)

94

In [8]:
#Consolidating all the recipes in a list
GialloZafferano_Corpus = []

for recipe in recipes:
    GialloZafferano_Corpus.append(get_ingredients(recipe))

In [9]:
GialloZafferano_Corpus

['spaghetti  g guanciale  g tuorli di uova medie  pecorino romano  g pepe nero q.b. per preparare gli spaghetti alla carbonara cominciate mettendo sul fuoco una pentola con lacqua salata per cuocere la pasta. nel frattempo eliminate la cotenna dal guanciale  e tagliatelo prima a fette e poi a striscioline spesse circa cm  . la cotenna avanzata potrà essere riutilizzata per insaporire altre preparazioni. versate i pezzetti in una padella antiaderente  e rosolate per circa  minuti a fiamma media fate attenzione a non bruciarlo altrimenti rilascerà un aroma troppo forte. nel frattempo tuffate gli spaghetti nellacqua bollente  e cuoceteli per il tempo indicato sulla confezione. intanto versate i tuorli in una ciotola   aggiungete anche la maggior parte del pecorino previsto dalla ricetta  la parte restante servirà per guarnire la pasta. insaporite con il pepe nero   amalgamate tutto con una frusta a mano  . aggiungete un cucchiaio di acqua di cottura per diluire il composto e mescolate  . 

# OpenTable Corpus

In this section we are going to create a web scraper to retrieve Italian restaurant menus from restaurants based in Italy listed on OpenTable (https://www.opentable.com/), an online restaurant-reservation service company. The website is in English but it often posts the menus in Italian of the restaurants listed.

In [13]:
def get_menus(url):
    myfile = requests.get(url)
    soup = BeautifulSoup(myfile.content)
    
    headers = soup.findAll("div", {"class": "menu-item-header__3xwnFL-n"})
    descriptions = soup.findAll("p", {"class": "menu-description__2HXkC4oE"})
    menu = headers + descriptions
    
    for w in range(0, len(menu)):   
        menu[w] = str(menu[w])                     #converting the objects to strings
        menu[w] = re.sub('<[^>]+>', '', menu[w])   #eliminaiting the scripts in between <>
        
    menu = [' '.join(w for w in menu)] 
    
    menu = listToString(menu)
    menu = menu.lower() #lowercasing every word
    
    menu = re.sub('[^a-zA-ZÀ-ÿ.\s]', '', menu) #removing all the numbers and special characters
    menu = menu.replace(".", "")

    return menu

get_menus("https://www.opentable.com/r/risotteria-italiana-milan?corrid=9cf72e53-c4ec-4acf-ba3a-6cf9a9ae8daa&p=2&sd=2020-03-31%2022%3A30")

'meraviglioso spinacino zafferano rosso di sera aglio e olio bosco battuta e riso tropico carbonara caramellato oca fassona cervo vitello kobe zebra struzzo renna tartare di manzo tartare di vitello tartare di cervo trittico di tartare fiori sfere di faraona caprese tiramisú cremoso al mango goloso cannolo scomposto zacapa edition negra  zacapa xo  zacapa royal  rum rum martiniqua  last ward   jhonnie walker blue lable  talisker  north lagavulin   cardhu   bulleit bourbon   bulleit rye  nikka from the barrel  nikka taketsuru  salsa di pomodoro burrata pesto in polvere spinacío coulis di ciliegia guanciale croccante zafferano ragù di cervo barbabietola crema di pecorino rucola frítta olio allaglio peperoncino funghi portobello salvia speck essiccato pesto battuta di manzo zucchina marinata al maracuja pesto perlage basilico guanciale croccante crema pecorino tuorlo lardo cipolla caramellata miele speck doca affumicato crema di funghi portobello olio al rosmarino perlage di aglio olio ev

In [14]:
menus = ["https://www.opentable.com/r/crudite-milano-milan", "https://www.opentable.com/r/risotteria-italiana-milan?corrid=9cf72e53-c4ec-4acf-ba3a-6cf9a9ae8daa&p=2&sd=2020-03-31%2022%3A30",
        "https://www.opentable.it/r/ristorante-tenuta-fasano-naples?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=55bed57d-8db4-4580-87f3-cb1b43638fc8&p=2&sd=2020-05-10%2019%3A00",
        "https://www.opentable.it/r/le-isole-marsala?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=55bed57d-8db4-4580-87f3-cb1b43638fc8&p=2&sd=2020-05-10%2019%3A00",
        "https://www.opentable.it/r/libra-cucina-evolution-bologna?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=d401c110-7094-45fc-82cf-843bbb04d13a&p=2&sd=2020-05-10+19%3A00",
        "https://www.opentable.it/r/terrazza-danieli-venice?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=8d103dfc-4f8d-4066-b5e4-2231f10d6b42&p=2&sd=2020-05-22+19%3A00",
        "https://www.opentable.it/r/giardino-dinverno-venice?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=8d103dfc-4f8d-4066-b5e4-2231f10d6b42&p=2&sd=2020-05-22+19%3A00",
        "https://www.opentable.it/r/savo-gourmet-pizzeria-genova?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=269cccf0-9ccb-48e0-a2d1-44c91ec8ad2f&p=2&sd=2020-05-22+19%3A00",
        "https://www.opentable.it/r/mi-garba-milan?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=c706d767-bac1-4a00-bea7-ac4856784522&p=2&sd=2020-05-22%2019%3A00",
        "https://www.opentable.it/r/ristorante-alle-langhe-milan?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=c706d767-bac1-4a00-bea7-ac4856784522&p=2&sd=2020-05-22%2019%3A00",
        "https://www.opentable.it/r/god-save-the-food-piave-milan?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=c706d767-bac1-4a00-bea7-ac4856784522&p=2&sd=2020-05-22%2019%3A00",
        "https://www.opentable.it/r/osteria-della-via-appia-milano?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=c706d767-bac1-4a00-bea7-ac4856784522&p=2&sd=2020-05-22%2019%3A00",
        "https://www.opentable.it/r/la-pizza-biscottata-gourmet-milan?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=c706d767-bac1-4a00-bea7-ac4856784522&p=2&sd=2020-05-22%2019%3A00",
        "https://www.opentable.it/r/caffe-and-bistrot-la-locanda-del-gatto-rosso-milan?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=c706d767-bac1-4a00-bea7-ac4856784522&p=2&sd=2020-05-22+19%3A00",
        "https://www.opentable.it/r/risotteria-italiana-milan?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=c706d767-bac1-4a00-bea7-ac4856784522&p=2&sd=2020-05-22%2019%3A00",
        "https://www.opentable.com/r/ristorante-tenuta-fasano-naples?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=737f3f4d-d9a4-4d36-b579-637277b91fc8&p=2&sd=2020-06-05%2019%3A00",
        "https://www.opentable.com/r/eolo-ristobar-torregrande?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=dce038f0-c94f-4e51-b38e-ab1ce866a49c&p=2&sd=2020-06-05%2019%3A00"]

In [15]:
len(menus)

17

In [16]:
#Consolidating all the recipes in a list
OpenTable_Corpus = []

for menu in menus:
    OpenTable_Corpus.append(get_menus(menu))

In [17]:
OpenTable_Corpus

['scontro tra titani non cè trippa per rane nostalgia canaglia sinfonia dautunno oro rosso ed è subito sera per ogni riccio bacco per bacco polp fiction al mercato baccalà miseria e nobiltà plateau plateau plage plateau royale red king sorprendimi sottobosco petaloso tonniamo insieme king crab astice peccato di gola lultimo bacio per incanto o per delizia chocolat mazzancolla vs pancia di maialino e il suo fondo crema di patata ratta e foglie di barbabietola trippa di rana pescatrice pecorino menta e cubotti di polenta fritta gratinati da tradizione capasanta scottata fois gras spuma di castagne e melograno speghettone il valentino bisque il suo corallo disidratato gambero rosso in purezza risotto al nero di seppia tagliatella di seppia marinata e riduzione di peperone arrosto chitarra ricci di mare e acciughe disidratate fusellone ragù di polpo e sponsali al vino rosso polpo croccante crema di tipinambur e scarola ripassata pescato casseruola di patate funghi e carciofi baccalà cardo 

# Tabula File Processor

In this section we are going to process and clean the menues we retrieved using Tabula (https://tabula.technology/). Tabula is a free tool for extracting data from PDF files into CSV and Excel files. We retrieved the menus in PDF format from some Italian restaurants based in Italy and then we converted the retrieved CSV files to text files. 

In [18]:
#Upload the retrieved menus from the directory
dir_base = r"C:\Users\andre\Tabula Menus"

def read_file(filename):
    input_file_text = open(filename , encoding='utf-8').read()
    return input_file_text

    
def read_directory_files(directory):
    file_texts = []
    files = [f for f in listdir(directory) if isfile(join(directory, f))]
    for f in files:
        file_text = read_file(join(directory, f))
        file_texts.append(file_text)
    return file_texts
    
Tabula_Corpus = read_directory_files(dir_base)
print(Tabula_Corpus)

['"Focaccia alla zucca con burrata, pesto e taggiasche"\n"Taglierino Parmigiano: Tagliere con Culaccia, scaglie di Parmigiano Reggiano 20"\n"mesi, focaccia integrale e marmellata di pere"\nTartare di tonno e pistacchi\nPolenta ramata porcini e fonduta\n"Vellutata di patate dolci, zucca e castagne croccanti"\n"Carpaccio di ricciola guacamole, mango e soia"\nBattuta di Fassona con porro croccante e maionese ai capperi\n"Tagliere mini bocconcino: Bocconcini di Culatta di Lagrimone Salumificio Vescovi,"\nscaglie di Parmigiano Reggiano 24 mesi con pan-focaccia alla zucca (per una\npersona)\n"Tagliere medio: Tagliere con Culatta di Lagrimone Salumificio Vescovi, bresaola di"\n"Fassona, scaglie di Parmigiano Reggiano 24 mesi, ricotta salata, composta di pere e"\nfocaccia alla zucca\n"Tagliere premium: Tagliere con Culatta di Lagrimone Salumificio Vescovi, bresaola di"\n"Fassona, scaglie di Parmigiano Reggiano 24 mesi, ricotta salata, burrata di Andria,"\n"chips di patate dolci, arancini di qu

In [19]:
#Exploring the object tabula_corpus
print(type(Tabula_Corpus))
print(len(Tabula_Corpus))

<class 'list'>
17


In [20]:
#creating a function menu_cleaner to clean the menus

def menu_cleaner(menu): 
    menu = menu.replace('\n', ' ')
    menu = menu.replace('"', '')
    menu = re.sub('[^a-zA-ZÀ-ÿ.\s]', '', menu)
    menu = menu.lower()
    
    return menu

In [21]:
#appending all the cleaned menus to a new list tabula_corpus_cleaned
Tabula_Corpus_Cleaned = []

for m in Tabula_Corpus:
    m = menu_cleaner(m)
    
    Tabula_Corpus_Cleaned.append(m)
    
Tabula_Corpus_Cleaned 

['focaccia alla zucca con burrata pesto e taggiasche taglierino parmigiano tagliere con culaccia scaglie di parmigiano reggiano  mesi focaccia integrale e marmellata di pere tartare di tonno e pistacchi polenta ramata porcini e fonduta vellutata di patate dolci zucca e castagne croccanti carpaccio di ricciola guacamole mango e soia battuta di fassona con porro croccante e maionese ai capperi tagliere mini bocconcino bocconcini di culatta di lagrimone salumificio vescovi scaglie di parmigiano reggiano  mesi con panfocaccia alla zucca per una persona tagliere medio tagliere con culatta di lagrimone salumificio vescovi bresaola di fassona scaglie di parmigiano reggiano  mesi ricotta salata composta di pere e focaccia alla zucca tagliere premium tagliere con culatta di lagrimone salumificio vescovi bresaola di fassona scaglie di parmigiano reggiano  mesi ricotta salata burrata di andria chips di patate dolci arancini di quinoa composta di pere e focaccia alla zucca i nostri piatti al bacio

# Background Corpora

In this section we are going to create the final dataset, or background corpora. In order to do that, we are going to merge the three list together, convert the final list to a dictionary and create a file in .json format.

In [22]:
#Consolidating the three lists in one 
Background_Corpora = GialloZafferano_Corpus + OpenTable_Corpus + Tabula_Corpus_Cleaned
Background_Corpora

['spaghetti  g guanciale  g tuorli di uova medie  pecorino romano  g pepe nero q.b. per preparare gli spaghetti alla carbonara cominciate mettendo sul fuoco una pentola con lacqua salata per cuocere la pasta. nel frattempo eliminate la cotenna dal guanciale  e tagliatelo prima a fette e poi a striscioline spesse circa cm  . la cotenna avanzata potrà essere riutilizzata per insaporire altre preparazioni. versate i pezzetti in una padella antiaderente  e rosolate per circa  minuti a fiamma media fate attenzione a non bruciarlo altrimenti rilascerà un aroma troppo forte. nel frattempo tuffate gli spaghetti nellacqua bollente  e cuoceteli per il tempo indicato sulla confezione. intanto versate i tuorli in una ciotola   aggiungete anche la maggior parte del pecorino previsto dalla ricetta  la parte restante servirà per guarnire la pasta. insaporite con il pepe nero   amalgamate tutto con una frusta a mano  . aggiungete un cucchiaio di acqua di cottura per diluire il composto e mescolate  . 

In [23]:
len(Background_Corpora) #we can add more files to the corpora, I think 84 is enough to get started but then we can get more

128

In [24]:
#converting tabula_corpus_cleaned from a list to a dictionary
Background_Corpora = dict(enumerate(Background_Corpora))
Background_Corpora

{0: 'spaghetti  g guanciale  g tuorli di uova medie  pecorino romano  g pepe nero q.b. per preparare gli spaghetti alla carbonara cominciate mettendo sul fuoco una pentola con lacqua salata per cuocere la pasta. nel frattempo eliminate la cotenna dal guanciale  e tagliatelo prima a fette e poi a striscioline spesse circa cm  . la cotenna avanzata potrà essere riutilizzata per insaporire altre preparazioni. versate i pezzetti in una padella antiaderente  e rosolate per circa  minuti a fiamma media fate attenzione a non bruciarlo altrimenti rilascerà un aroma troppo forte. nel frattempo tuffate gli spaghetti nellacqua bollente  e cuoceteli per il tempo indicato sulla confezione. intanto versate i tuorli in una ciotola   aggiungete anche la maggior parte del pecorino previsto dalla ricetta  la parte restante servirà per guarnire la pasta. insaporite con il pepe nero   amalgamate tutto con una frusta a mano  . aggiungete un cucchiaio di acqua di cottura per diluire il composto e mescolate 

In [25]:
#converting Background_Corpora to .json
#this should create a .json file called Background Corpora in your computer, in your Python directory

with open('Background Corpora.json', 'w') as file:
    json.dump(Background_Corpora, file)

### Testing the corpora: 

This code won't be in the final project, it is just to see if we can upload the .json file properly and we can work on it

In [ ]:
with open('Background Corpora.json', 'r') as j:
    Background_Corpora_Json = json.load(j)
    print(Background_Corpora_Json)

In [ ]:
Background_Corpora_Json.keys()

In [ ]:
Background_Corpora_Json.get('0')

In [ ]:
Background_Corpora_Json.get('19')